# Parsers

In [ ]:
import numpy as np
import pandas as pd

from src.parsers import parse_reporting_info, parse_closed_banks

In [ ]:
REPORTING_DATA_FOLDER = 'parsed_data/bank_reporting/'
CLOSED_BANKS_DATA_FOLDER = 'parsed_data/closed_banks/'

# Список из ID страничек и соответствующие им названия переменных. 
ID_list = ['10','30','25','20','40','50','60','500','1000','1100','1200','1300','1400','1500','1550',\
           '1600','1700','1800','200','260','300','360','120','70','110','130','140','160','400','401']

ID_names_list = ['na','ni','c_123','c_134','cp','LLPGL','dep_f','dep_b','ROA','ROE','LLP','IR','LCP','fr_na',\
                 'fr','H1','H2','H3','c_f','LLP_c_f','c_b','LLP_c_b','ibl','sec_tot','liquid','sec','bond','oth_cap',\
                 'retail','retail_fr']

### 1. Bank reporting info

In [ ]:
# Сгенерируем лист из дат, которые необходимо пробежать.
# февраль 17 - март 08  (2017-02-01 до  2008-03-01)
years = [str(item) for item in range(2010,2018)]
months = [str(item) for item in range(1,13)]
months2 = [ ]
for item in months:
    if len(item)==1:
        item = '0'+item
    months2.append(item)
dats = [ ]
for item in years:
    for jtem in months2:
        dats.append(item + '-' + jtem + '-01')

In [ ]:
parse_reporting_info(ID_list, dats, path=REPORTING_DATA_FOLDER)
parse_closed_banks(path=CLOSED_BANKS_DATA_FOLDER)

In [ ]:
# Для начала неплохо было бы выявить список из уникальных банков! 

# Функция, которая обхелиняет два лоиста по уникалтным названиям 
# Она делает пересечение двух множеств 
def union(a, b):
    return list(set(a) | set(b))

def Uniq_banks(frames):
    uniq_banks = list(frames[0]['Лицензия_банк'])
    for item in ID_list:
        uniq_banks = union(uniq_banks,item)
    return(uniq_banks)

# Сохраним ко всему прочему информацию о том какие банки уникальны.
uniq_banks = Uniq_banks(frames)
un_banks = pd.DataFrame()
un_banks['Уникальные банки'] = uniq_banks

# Делаем список из датафреймов, которые мы подгружаем из папки. Соединим все показатели в один датафрейм
frames = [pd.read_csv(REPORTING_DATA_FOLDER + strtem +'.csv', header=0, sep=',', encoding="cp1251") 
          for strtem in ID_list]
df_all = frames[0].set_index(['лицензия', 'дата'])
    
i = 1
while i <= len(frames):
    df_tmp = pd.DataFrame(frames[i])
    df_all = df_all.join(df_tmp.set_index(['лицензия', 'дата']), how = 'outer')
    df_all.drop_duplicates(keep = 'first', inplace =True)
    i = i + 1

In [ ]:
df_all.to_csv(REPORTING_DATA_FOLDER + 'reporting_all.csv', sep=',',header=True,index=True)
un_banks.to_csv(REPORTING_DATA_FOLDER + 'unique_banks.csv',sep='\t',header=True,index=False)

### 2. Closed banks info

In [ ]:
parse_closed_banks(path=CLOSED_BANKS_DATA_FOLDER, mainpage='http://www.banki.ru')

In [ ]:
closed_banks = pd.read_csv('closed_banks.csv', header=0, sep=',', encoding="cp1251")
closed_banks['ликвдата'] = pd.to_datetime(closed_banks['ликвдата'])

# Причины по ликвидированным банкам: 
likvid_data = closed_banks[closed_banks['статус']=='ликв.']
otozv_data = closed_banks[closed_banks['статус']=='отозв.']

likvid_data.to_csv(CLOSED_BANKS_DATA_FOLDER + 'likvid_data.csv',sep='\t',header=True,index=False)
otozv_data.to_csv(CLOSED_BANKS_DATA_FOLDER + 'otozv_data.csv',sep='\t',header=True,index=False)

### 3. Other data

Other data was already in the form of .csv at Central Bank website

# Объединим все показатели в один датафрей

## Данные, которые нужны для модели

Сопоставление критериев и их ID:

+ Активы нетто: na     10
+ Чистая прибыль ni    30
(excel) + Капитал (по форме 123) c_123     25
(excel) + Капитал (по форме 134) c_134     20
+ Кредитный портфель credit_portf   40
+ Просроченная задолженность в кредитном портфеле sunk_credit_portf   50
+ Вклады физических лиц retail_deposit      60
+ Средства предприятий и организаций organization_deposit   500
+ Рентабельность активов-нетто ROA    1000
+ Рентабельность капитала ROE         1100 
     + Уровень просроченной задолженности по кредитному портфелю sunk_credit_perc   1200
     + Уровень резервирования по кредитному портфелю reserv_credit_perc    1300
     + Уровень обеспечения кредитного портфеля залогом имущества zalog_credit_perc      1400
     + Валютный оборот к активам-нетто foreign_na            1500
     + Н1 H1    1600
     + Н2 H2    1700
     + Н3 H3    1800
+ Кредиты физическим лицам retail_credit    200
+ Просроченная задолженность по кредитам физическим лицам sunk_retail_credit   260
+ Кредиты предприятиям и организациям organization_credit   300
+ Просроченная задолженность по кредитам предприятиям и организациям sunk_organization_credit_perc   360
+ Выданные МБК ibl   120

+ вложения в ценные бумаги:sec_tot 70 = подверженность рискам на фин рынках
+ высоколиквидные активы: liquid 110
+ вложения в акции sec 130
+ вложения в облигации bond 140
+ вложения в капиталы других организаций oth_cap 160
+ вклады физических лиц retail 400
     + вклады физических лиц оборот retail_deposit_fr 401

На Мар-шаге мы скачали все необходимые нам показатели с сайта в кучу маленьких csv-табличек, которые нам теперь нужно переработать в одну большую таблицу, которую мы будем впоследствии использовать при оценки моделей. Самого по себе Reduce-шага здесь не будет. Будет Panel - шаг!

Мы хотим, чтобы в итоговой таблице оформление было таким, чтобы было удобно в python работать с панельными данными. Итоговая таблица должна идти вот в таком формате:



| Банк   | Год   | Показатель 1 | Показатель 2 | Показатель 3 |
|--------|-------|--------------|--------------|--------------|
| Банк 1 | Год 1 |              |              |              |
| Банк 1 | Год 2 |              |              |              |
| Банк 2 | Год 1 |              |              |              |
| Банк 2 | Год 2 |              |              |              |

# ЭТАП 2

In [14]:
# Подгружаем списки из банков.
uniq_banks = list(pd.read_csv(REPORTING_DATA_FOLDER + 'unique_banks.csv',sep='\t',
                              header = 0, encoding="cp1251").iloc[:,0])
uniq_banks2 = [re.sub("\D",'',item) for item in uniq_banks]

print(len(uniq_banks))

# Подгружаем данные по всякой отозванной фигне. 
likvid_data = pd.read_csv(CLOSED_BANKS_DATA_FOLDER + 'likvid_data.csv',sep='\t',header=0, encoding="cp1251")
otozv_data = pd.read_csv(CLOSED_BANKS_DATA_FOLDER + 'otozv_data.csv',sep='\t',header=0, encoding="cp1251")
print(likvid_data.shape[0])
print(otozv_data.shape[0])

# Подгружаем табличку с данными.
bank_frame = pd.read_csv(REPORTING_DATA_FOLDER + 'reporting_all.csv',,sep=',',header=0, encoding="cp1251")
#uniq_banks = set(bank_frame['Банк'])

1181
503
2047


C:\Users\imaksimov002\AppData\Local\Continuum\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Получим номер лицензии

In [300]:
# функция для выделения номера лицензии из строки
def get_license(a):
    license = a.split('№ ')[1].split(',')[0]
    return(license)

In [320]:
bank_frame['лицензия'] = 0
p = np.array(range(bank_frame.shape[0]))
for i in range(80287):
    p[i] = get_license(bank_frame.iloc[i, 0])
bank_frame['лицензия'] = p

In [324]:
bank_frame.to_csv('all_ok_v5' + '.csv',sep=',',header=True, index=False)

# Норматив ликвидности банка Н3 

In [508]:
liquidity = pd.read_csv('liquidity_2.csv',sep=',', header = 0,  encoding="cp1251")

In [509]:
liquidity.dropna(axis=0, how='any', inplace = True)
liquidity.columns = ['drop', 'лицензия', 'N3', 'дата']
liquidity[['лицензия']] = liquidity[['лицензия']].astype(int)
liquidity['дата'] = pd.to_datetime(liquidity['дата'])

In [510]:
liquidity.drop(['drop'], inplace = True, axis = 1)

In [511]:
liquidity.head(5)

,лицензия,N3,дата
0,1,73.16,2010-06-01
1,67,63.10,2010-06-01
2,85,90.30,2010-06-01
3,96,143.33,2010-06-01
4,101,147.25,2010-06-01


# Обработка данных по закрытым банкам

In [523]:
#closed_banks[['Лицензия']] = closed_banks[['Лицензия']].astype(int)
closed_banks['ликвдата'] = pd.to_datetime(closed_banks['ликвдата'])

In [530]:
# Данные скачались немного криво: надо поменять день и месяц местами
closed_banks['дата'] = 0
for u in range(closed_banks.shape[0] - 1):
    y = closed_banks.iloc[u, 1].year
    m = closed_banks.iloc[u, 1].month
    if len(str(m)) == 1:
        m = '-0'+ str(m)
    else:
        m = '-' + str(m)
    d = '-01'
    date = str(y) + m + d
    date = pd.to_datetime(date)
    closed_banks.iloc[u, 2] = date

In [532]:
closed_banks.drop('ликвдата', inplace = True, axis = 1)

In [534]:
closed_banks.columns = ['лицензия','дата']

In [536]:
closed_banks['default'] = 1

# Макро показатели

In [496]:
macro = pd.read_csv('macro_1.csv',sep=',', header = 0,  encoding="cp1251")

In [497]:
macro['Date'] = pd.to_datetime(macro['Date'])

In [499]:
macro.columns = ['drop','дата', 'M2', 'M2_SA', 'INF_SA', 'NX', 'NX_log', 'Nx_growth', 'usd_rub', 'usd_rub_std',\
                 'micex', 'micex_std', 'miacr', 'miacr_std', 'miacr_amount', 'net_foreign_assets',\
                 'net_gov_debt', 'other_fin_debt', 'retail_debt', 'stocks_capital', 'i_retail_credit', 'i_retail_debit',\
                 'i_retail_spread']

In [501]:
macro.drop(['drop'], inplace = True, axis = 1)

In [502]:
macro.head(5)

,дата,M2,M2_SA,INF_SA,NX,NX_log,Nx_growth,usd_rub,usd_rub_std,micex,...,miacr_std,miacr_amount,net_foreign_assets,net_gov_debt,other_fin_debt,retail_debt,stocks_capital,i_retail_credit,i_retail_debit,i_retail_spread
0,2009-12-01,14967.6,14477.677786,0.020000,13107,9.480902,0,29.60000,0.180000,1433.648,...,1.181512,2643995,14099099,-6769343,516858,3959248,6004665,17.1,6.0,11.1
1,2010-01-01,15267.6,14477.677786,0.020000,15835,9.669978,2728,29.83873,0.374754,1433.648,...,0.724981,2643995,13942674,-5174038,564379,3932605,6219422,17.1,6.0,11.1
2,2010-02-01,14904.1,14875.445322,0.027475,14855,9.606092,-980,30.15797,0.177248,1351.718,...,0.330918,2643995,14168103,-5439869,529987,3913803,6279331,17.2,6.2,11.0
3,2010-03-01,15236.4,15247.121207,0.024986,14556,9.585759,-299,29.55940,0.238446,1410.415,...,0.359785,2643995,14039665,-5008998,508111,3898961,6283624,17.1,6.0,11.1
4,2010-04-01,15639.4,15570.868119,0.021233,13789,9.531626,-767,29.19324,0.158573,1478.745,...,0.246044,2643995,13815998,-4916261,507036,3910081,6347867,17.1,6.1,11.0


# Капитал банков

In [447]:
capital = pd.read_csv('capital_1.csv',sep=',', header = 0,  encoding="cp1251")

In [450]:
capital.columns = ['drop', 'date', 'bank', 'license', 'capital', 'msk_spb']
capital.drop(['drop'], inplace = True, axis = 1)

In [451]:
capital.dropna(axis=0, how='any', inplace = True)
capital[['license']] = capital[['license']].astype(int)
capital['date'] = pd.to_datetime(capital['date'])

In [452]:
capital.columns = ['дата', 'банк', 'лицензия', 'capital', 'msk_spb']

In [746]:
capital.tail(5)

,дата,банк,лицензия,capital,msk_spb
74869,2017-11-01,Элплат,3535,18995.0,0
75434,2017-12-01,Элплат,3535,18995.0,0
74277,2017-10-01,Р-Инкас,3536,102845.0,1
74849,2017-11-01,Р-Инкас,3536,97707.0,1
75415,2017-12-01,Р-Инкас,3536,90689.0,1


# Join всего в один датафрейм

In [455]:
bank_frame[['лицензия']] = bank_frame[['лицензия']].astype(int)
bank_frame['дата'] = pd.to_datetime(bank_frame['дата'])

In [471]:
capital.sort_values(by = ['лицензия', 'дата'], inplace = True)
bank_frame.sort_values(by = ['лицензия', 'дата'], inplace = True)
liquidity.sort_values(by = ['лицензия', 'дата'], inplace = True)

In [458]:
df = bank_frame.set_index(['лицензия', 'дата']).join(capital.set_index(['лицензия', 'дата']), how = 'inner')

In [477]:
df_1 = df.join(liquidity.set_index(['лицензия', 'дата']), how = 'left')

In [493]:
df_1['ИД_1800'] = df_1['ИД_1800'].fillna(0).astype(int)
df_1['N3'] = df_1['N3'].fillna(0).astype(int)

ValueError: invalid literal for int() with base 10: '1 372,32'

In [503]:
df_2 = df.join(macro.set_index(['дата']), how = 'inner')

In [512]:
df_3 = df_2.join(liquidity.set_index(['лицензия', 'дата']), how = 'left')

In [563]:
# Очистка данных
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-К', '')
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-Д', '')
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-Р', '')
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-K', '')
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-к', '')
closed_banks = closed_banks.iloc[ :(closed_banks.shape[0] - 2), :]
closed_banks[['лицензия']] = closed_banks[['лицензия']].astype(int)
closed_banks['дата'] = pd.to_datetime(closed_banks['дата'])

In [578]:
closed_banks.sort_values(by = ['лицензия', 'дата'], inplace = True)

C:\Users\imaksimov002\AppData\Local\Continuum\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [673]:
df_4 = df_3.join(closed_banks.set_index(['лицензия', 'дата']), how = 'left')

In [626]:
df_4.shape

(74633, 54)

In [674]:
df_4['default'] = df_4['default'].fillna(0)
df_4.drop(['Лицензия_банк'], inplace = True, axis = 1)

In [593]:
df_4.to_csv('final_data_3' + '.csv',sep=',',header=True, index=True)

In [591]:
df_4.head(5)

ИД_10 ИД_1000   ИД_110   ИД_120 ИД_130 ИД_140 ИД_160  \
лицензия дата                                                                 
0        2010-02-01  423 017      27  112 770   60 000      0      0      0   
         2010-03-01  498 411      75  172 628   90 000      0      0      0   
         2010-04-01  571 220      54  211 860   90 000      0      0      0   
         2010-05-01  523 027      41  159 970   90 000      0      0      0   
         2010-06-01  473 713      31  131 782  135 000      0      0      0   

                    ИД_260 ИД_30   ИД_300   ...    net_foreign_assets  \
лицензия дата                               ...                         
0        2010-02-01      0   102  225 000   ...              14168103   
         2010-03-01      0   569  225 000   ...              14039665   
         2010-04-01    997   651  227 287   ...              13815998   
         2010-05-01    481   677  227 130   ...              14067700   
         2010-06-01    485   644  191 306   ...              14454661   

                    net_gov_debt other_fin_debt retail_debt stocks_capital  \
лицензия дата                                                                
0        2010-02-01     -5439869         529987     3913803        6279331   
         2010-03-01     -5008998         508111     3898961        6283624   
         2010-04-01     -4916261         507036     3910081        6347867   
         2010-05-01     -5051224         507117     3955282        6383875   
         2010-06-01     -4878647         496365     4014144        6356903   

                    i_retail_credit i_retail_debit i_retail_spread  N3  \
лицензия дата                                                            
0        2010-02-01            17.2            6.2            11.0 NaN   
         2010-03-01            17.1            6.0            11.1 NaN   
         2010-04-01            17.1            6.1            11.0 NaN   
         2010-05-01            17.1            6.0            11.1 NaN   
         2010-06-01            17.2            5.8            11.4 NaN   

                     default  
лицензия дата                 
0        2010-02-01      0.0  
         2010-03-01      0.0  
         2010-04-01      0.0  
         2010-05-01      0.0  
         2010-06-01      0.0  

[5 rows x 53 columns]

In [676]:
# очистка данных:

gg = lambda x: x.replace('?','-') # ? -> -
pp = lambda y: y.replace(' ', '') # убратб пробелы
cc = lambda z: z.replace(',','.') # , -> .

item = ['10', '1000','110','120','130', '140','160', '260', '30', '300', '360','40', '400', '50', '500', '60', '70', '1100', '200','1300', '1400', '1500', '401', '1800', '1700','1600']
for i in item:
    a = "ИД_" + i
    df_4[a] = df_4[a].astype(str)
    df_4[a] = df_4[a].apply(gg)
    df_4[a] = df_4[a].apply(pp)
    df_4[a] = df_4[a].apply(cc)

In [677]:
# часть данных оказалась в долях, а не в %: исправим это
df_4['ИД_1000'] = df_4['ИД_1000'].astype(float) + (df_4['ИД_1000'].astype(float) < 0)*df_4['ИД_1000'].astype(float)*99
df_4['ИД_1100'] = df_4['ИД_1100'].astype(float) + (df_4['ИД_1100'].astype(float) < 0)*df_4['ИД_1100'].astype(float)*99

In [679]:
# перевести в числовой тип
item = ['10', '1000','110','120','130', '140','160', '260', '30', '300', '360','40', '400', '50', '500', '60', '70', '1100', '200','1300', '1400', '1500', '401', '1800', '1700','1600']
for i in item:
    a = "ИД_" + i
    df_4[a] = df_4[a].astype(float)

In [651]:
df_4.to_csv('final_data_5' + '.csv',sep=',',header=True, index=True)

# Обработка пропусков (NaN)

In [680]:
df_5 = df_4

In [691]:
df_5.dropna(thresh = 16, subset = ['ИД_10', 'ИД_1000','ИД_110','ИД_120','ИД_130', 'ИД_140','ИД_160', 'ИД_260',\
                                  'ИД_30', 'ИД_300', 'ИД_360','ИД_40', 'ИД_400', 'ИД_50', 'ИД_500', 'ИД_60',\
                                  'ИД_70', 'ИД_1100'],
           inplace = True)

In [731]:
col = df_5.columns

'ИД_10'

In [704]:
# Сохраним данные о названиях банков
bank_names = df_5.iloc[:, 27]
bank_names.to_csv('bank_names' + '.csv',sep=',',header=True, index=True)

In [705]:
# уберу ненужные столбцы
df_5.drop(['лиц_дат', 'банк'], inplace = True, axis = 1)

In [721]:
# Трюк, чтобы индексы стали столбцами
df_5.to_csv('final_data_6' + '.csv',sep=',',header=True, index=True)
df_6 = pd.read_csv('final_data_6.csv',sep=',', header = 0,  encoding="cp1251")

# Отдельно запишем индексы
lic = df_6['лицензия']
dat = df_6['дата']

In [726]:
# Заполним пропуски медианой
from sklearn.preprocessing import Imputer
imr = Imputer(missing_values = 'NaN', strategy = 'median', axis = 0)
imr.fit(df_5)
df_imp = df_5
df_imp = imr.transform(df_5.values)

In [735]:
# Добавим индексы
df_imp = pd.DataFrame(df_imp, columns = col)
df_imp['лицензия'] = lic
df_imp['дата'] = dat
df_imp = df_imp.set_index(['лицензия', 'дата'])
df_imp.head(5)

ИД_10  ИД_1000    ИД_110    ИД_120  ИД_130  ИД_140  \
лицензия дата                                                                
0        2010-02-01  423017.0     27.0  112770.0   60000.0     0.0     0.0   
         2010-03-01  498411.0     75.0  172628.0   90000.0     0.0     0.0   
         2010-04-01  571220.0     54.0  211860.0   90000.0     0.0     0.0   
         2010-05-01  523027.0     41.0  159970.0   90000.0     0.0     0.0   
         2010-06-01  473713.0     31.0  131782.0  135000.0     0.0     0.0   

                     ИД_160  ИД_260  ИД_30    ИД_300   ...     \
лицензия дата                                          ...      
0        2010-02-01     0.0     0.0  102.0  225000.0   ...      
         2010-03-01     0.0     0.0  569.0  225000.0   ...      
         2010-04-01     0.0   997.0  651.0  227287.0   ...      
         2010-05-01     0.0   481.0  677.0  227130.0   ...      
         2010-06-01     0.0   485.0  644.0  191306.0   ...      

                     net_foreign_assets  net_gov_debt  other_fin_debt  \
лицензия дата                                                           
0        2010-02-01          14168103.0    -5439869.0        529987.0   
         2010-03-01          14039665.0    -5008998.0        508111.0   
         2010-04-01          13815998.0    -4916261.0        507036.0   
         2010-05-01          14067700.0    -5051224.0        507117.0   
         2010-06-01          14454661.0    -4878647.0        496365.0   

                     retail_debt  stocks_capital  i_retail_credit  \
лицензия дата                                                       
0        2010-02-01    3913803.0       6279331.0             17.2   
         2010-03-01    3898961.0       6283624.0             17.1   
         2010-04-01    3910081.0       6347867.0             17.1   
         2010-05-01    3955282.0       6383875.0             17.1   
         2010-06-01    4014144.0       6356903.0             17.2   

                     i_retail_debit  i_retail_spread     N3  default  
лицензия дата                                                         
0        2010-02-01             6.2             11.0  92.27      0.0  
         2010-03-01             6.0             11.1  92.27      0.0  
         2010-04-01             6.1             11.0  92.27      0.0  
         2010-05-01             6.0             11.1  92.27      0.0  
         2010-06-01             5.8             11.4  92.27      0.0  

[5 rows x 51 columns]

# Переименуем названия столбцов

In [736]:
df_imp.columns

Index(['ИД_10', 'ИД_1000', 'ИД_110', 'ИД_120', 'ИД_130', 'ИД_140', 'ИД_160',
       'ИД_260', 'ИД_30', 'ИД_300', 'ИД_360', 'ИД_40', 'ИД_400', 'ИД_50',
       'ИД_500', 'ИД_60', 'ИД_70', 'ИД_1100', 'ИД_200', 'ИД_1300', 'ИД_1400',
       'ИД_1500', 'ИД_401', 'ИД_1800', 'ИД_1700', 'ИД_1600', 'capital',
       'msk_spb', 'M2', 'M2_SA', 'INF_SA', 'NX', 'NX_log', 'Nx_growth',
       'usd_rub', 'usd_rub_std', 'micex', 'micex_std', 'miacr', 'miacr_std',
       'miacr_amount', 'net_foreign_assets', 'net_gov_debt', 'other_fin_debt',
       'retail_debt', 'stocks_capital', 'i_retail_credit', 'i_retail_debit',
       'i_retail_spread', 'N3', 'default'],
      dtype='object')

In [753]:
columns = ['net_assets', 'ROA', 'liquid', 'ibl', 'stocks', 'bond', 'oth_cap', 'sunk_retail_credit',\
          'NI', 'organization_credit', 'sunk_organization_credit', 'credit_portf', 'drop', 'sunk_credit_portf',\
          'organization_deposit', 'retail_deposit', 'security_tot', 'ROE', 'retail_credit', 'reserv_credit_perc',\
          'zalog_credit_perc', 'foreign_na_fr', 'retail_deposit_fr', 'N3', 'N2', 'N1', 'capital',\
           'msk_spb', 'M2', 'M2_SA', 'INF_SA', 'NX', 'NX_log', 'Nx_growth',\
           'usd_rub', 'usd_rub_std', 'micex', 'micex_std', 'miacr', 'miacr_std',\
           'miacr_amount', 'net_foreign_assets', 'net_gov_debt', 'other_fin_debt',\
           'retail_debt', 'stocks_capital', 'i_retail_credit', 'i_retail_debit',\
           'i_retail_spread', 'N3.1', 'default']

In [754]:
df_imp.columns = columns
df_imp.drop('drop', axis = 1, inplace = True)
df_imp.to_csv('FINAL_imp' + '.csv',sep=',',header=True, index=True)

In [758]:
# В столбце N3_early показатель Н3 до 2015-01-01, а в N3 - после. Надо их объединить

df_imp_1 = pd.read_csv('FINAL_imp.csv',sep=',', header = 0,  encoding="cp1251")
df_imp_1['N3'] = df_imp_1['N3']*(df_imp_1['дата'] > '2015-01-01') + df_imp_1['N3.1']*(df_imp_1['дата'] <= '2015-01-01')
df_imp_1.drop(['N3.1'], axis = 1, inplace = True)

In [759]:
# количество уникальных банков
len(np.unique(lic))

1049

In [760]:
df_imp_1.head(5)

,лицензия,дата,net_assets,ROA,liquid,ibl,stocks,bond,oth_cap,sunk_retail_credit,...,miacr_amount,net_foreign_assets,net_gov_debt,other_fin_debt,retail_debt,stocks_capital,i_retail_credit,i_retail_debit,i_retail_spread,default
0,0,2010-02-01,423017.0,27.0,112770.0,60000.0,0.0,0.0,0.0,0.0,...,2643995.0,14168103.0,-5439869.0,529987.0,3913803.0,6279331.0,17.2,6.2,11.0,0.0
1,0,2010-03-01,498411.0,75.0,172628.0,90000.0,0.0,0.0,0.0,0.0,...,2643995.0,14039665.0,-5008998.0,508111.0,3898961.0,6283624.0,17.1,6.0,11.1,0.0
2,0,2010-04-01,571220.0,54.0,211860.0,90000.0,0.0,0.0,0.0,997.0,...,2643995.0,13815998.0,-4916261.0,507036.0,3910081.0,6347867.0,17.1,6.1,11.0,0.0
3,0,2010-05-01,523027.0,41.0,159970.0,90000.0,0.0,0.0,0.0,481.0,...,2643995.0,14067700.0,-5051224.0,507117.0,3955282.0,6383875.0,17.1,6.0,11.1,0.0
4,0,2010-06-01,473713.0,31.0,131782.0,135000.0,0.0,0.0,0.0,485.0,...,2643995.0,14454661.0,-4878647.0,496365.0,4014144.0,6356903.0,17.2,5.8,11.4,0.0


In [761]:
df_imp_1.to_csv('FINAL_OK_imp' + '.csv',sep=',',header=True, index=True)